# 전체 json 파일을 개별 json 파일로 분리

In [ ]:
import json
import os
import re

# 원본 JSON 파일 경로
input_file = 'path_to_your_file.json'

# 개별 JSON 파일을 저장할 디렉토리
output_dir = 'output_json_files'

# 출력 디렉토리가 없으면 생성
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# 원본 JSON 파일 읽기
with open(input_file, 'r', encoding='utf-8') as f:
    data = json.load(f)

# 각 이미지에 대해 개별 JSON 파일 생성
for item in data:
    # 이미지 이름 추출
    ocr_path = item.get("ocr", "")
    image_name_match = re.search(r'corrected_CAM_\d{14}_\d{1,2}_warped', ocr_path)
    
    if image_name_match:
        image_name = image_name_match.group(0)
        # 개별 JSON 파일 경로 설정
        output_file = os.path.join(output_dir, f'{image_name}.json')
        
        # 개별 JSON 파일 저장
        with open(output_file, 'w', encoding='utf-8') as f_out:
            json.dump(item, f_out, ensure_ascii=False, indent=4)

print(f"각 이미지에 대한 JSON 파일이 '{output_dir}' 디렉토리에 저장되었습니다.")


# 원하는 json 파일 형식으로 변환

In [25]:
import json
import os
import re
import stat

# JSON 파일이 들어있는 폴더 경로
folder_path = 'content/annotation_combine'

# 원본 JSON 파일 경로
#input_file = 'content/corrected_CAM_20231019083531_1_warped.json'


# 폴더 내 모든 파일에 대해 작업 수행
for filename in os.listdir(folder_path):
    if filename.endswith('.json'):
        input_file = os.path.join(folder_path, filename)
        
        # 원본 JSON 파일 읽기
        try:
            with open(input_file, 'r', encoding='utf-8') as f:
                data = json.load(f)
            print(f"Loaded JSON file from {input_file}")
        except Exception as e:
            print(f"Failed to load JSON file {input_file}: {e}")
            continue

        # 변환된 JSON 데이터를 저장할 딕셔너리
        transformed_data = {}

        if isinstance(data, dict):
            ocr_path = data.get("ocr", "")
            image_name_match = re.search(r'corrected_CAM_\d{14}_\d{1,2}_warped', ocr_path)

            if image_name_match:
                image_name = image_name_match.group(0)
                print(f"Processing image: {image_name}")

                # original_width와 original_height를 bbox에서 추출
                try:
                    original_width = data['bbox'][0]['original_width']
                    original_height = data['bbox'][0]['original_height']
                except (KeyError, IndexError) as e:
                    print(f"Error extracting original dimensions from {input_file}: {e}")
                    continue

                # objects 리스트 생성
                objects = []
                for bbox, label in zip(data['bbox'], data['label']):
                    try:
                        label_index = data['bbox'].index(bbox)
                        transcription = data['transcription'][label_index]
                        obj = {
                            "x": bbox['x'],
                            "y": bbox['y'],
                            "width": bbox['width'],
                            "height": bbox['height'],
                            "transcription": transcription,
                            "label": label['labels'][0]
                        }
                        objects.append(obj)
                    except KeyError as e:
                        print(f"Error creating object in {input_file}: {e}")
                        continue

                transformed_data = {
                    "file_name": image_name,
                    "id": data['id'],
                    "original_width": original_width,
                    "original_height": original_height,
                    "objects": objects
                }

        # 변환된 JSON 데이터 확인
        print(json.dumps(transformed_data, ensure_ascii=False, indent=4))

        # 변환된 JSON 데이터를 기존 파일에 덮어쓰기
        try:
            with open(input_file, 'w', encoding='utf-8') as f_out:
                json.dump(transformed_data, f_out, ensure_ascii=False, indent=4)
            print(f"File '{input_file}' has been updated with transformed data.")
        except Exception as e:
            print(f"Failed to update file {input_file}: {e}")

print(f"모든 JSON 파일이 변환되었습니다.")


Loaded JSON file from content/annotation_combine\corrected_CAM_20230916074132_1_warped.json
Processing image: corrected_CAM_20230916074132_1_warped
{
    "file_name": "corrected_CAM_20230916074132_1_warped",
    "id": 363,
    "original_width": 578,
    "original_height": 505,
    "objects": [
        {
            "x": 5.817782656421515,
            "y": 0.6281857603982133,
            "width": 9.001097694840833,
            "height": 7.286954820619274,
            "transcription": "F-",
            "label": "terminal_number"
        },
        {
            "x": 21.62458836443469,
            "y": 0.8794600645574986,
            "width": 28.649835345773877,
            "height": 7.789503428937845,
            "transcription": "A6-별05",
            "label": "terminal_number"
        },
        {
            "x": 70.03293084522502,
            "y": 4.397300322787493,
            "width": 17.45334796926454,
            "height": 7.03568051645999,
            "transcription": "행당(대)",
  

# 기존에 존재하던 json 파일들 이름 변경 (CAM_ -> CAM)

In [6]:
import os
import re

# 개별 JSON 파일이 저장된 디렉토리
directory = 'content/annotation_combine'

# 디렉토리에 있는 모든 파일을 순회
for filename in os.listdir(directory):
    # JSON 파일만 처리
    if filename.endswith('.json'):
        # 정규 표현식을 사용하여 이미지 이름이 조건과 일치하는지 확인
        image_name_match = re.search(r'corrected_CAM_\d{14}_\d{1,2}_warped', filename)
        
        if image_name_match:
            # CAM_을 CAM 로 대체
            new_filename = filename.replace('CAM_', 'CAM ')
            # 파일 경로 설정
            old_file_path = os.path.join(directory, filename)
            new_file_path = os.path.join(directory, new_filename)
            # 파일 이름 변경
            os.rename(old_file_path, new_file_path)
            print(f'Renamed: {filename} -> {new_filename}')

print("파일 이름 변경이 완료되었습니다.")


Renamed: corrected_CAM_20230916074132_1_warped.json -> corrected_CAM 20230916074132_1_warped.json
Renamed: corrected_CAM_20230916074334_1_warped.json -> corrected_CAM 20230916074334_1_warped.json
Renamed: corrected_CAM_20230916074352_1_warped.json -> corrected_CAM 20230916074352_1_warped.json
Renamed: corrected_CAM_20230916074557_1_warped.json -> corrected_CAM 20230916074557_1_warped.json
Renamed: corrected_CAM_20230916074622_1_warped.json -> corrected_CAM 20230916074622_1_warped.json
Renamed: corrected_CAM_20230916075108_1_warped.json -> corrected_CAM 20230916075108_1_warped.json
Renamed: corrected_CAM_20230916075536_1_warped.json -> corrected_CAM 20230916075536_1_warped.json
Renamed: corrected_CAM_20230916080051_1_warped.json -> corrected_CAM 20230916080051_1_warped.json
Renamed: corrected_CAM_20230916080129_1_warped.json -> corrected_CAM 20230916080129_1_warped.json
Renamed: corrected_CAM_20230916080134_12_warped.json -> corrected_CAM 20230916080134_12_warped.json
Renamed: corrected